In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/Users/hirofumi48/162348.github.io/posts/2025/PDMP"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| eval: false
#| code-fold: true

using PDMPFlux, Plots

@inline function ∇U(x::AbstractVector)
  return x
end

dim = 2
sampler = BPS(dim, ∇U; refresh_rate=0.0)

N_sk, xinit, vinit = 1_000, randn(dim), randn(dim)
vinit = vinit ./ sqrt(sum(vinit.^2))

output_BPS = sample_skeleton(sampler, N_sk, xinit, vinit, seed=20250428)
x_BPS = output_BPS.X
v_BPS = output_BPS.V
t_BPS = output_BPS.t

anim_traj(output_BPS, 100; plot_start=50, filename="BPS.gif", color="#E95420", scatter_color="#78C2AD", background="#F0F1EB", title="BPS without Refreshment", xlabel="", ylabel="", axis=false)

In [3]:
#| eval: false
#| code-fold: true

using PDMPFlux, LinearAlgebra

const γ = 10

@inline function ∇U(x::AbstractVector)
  return [γ,1/γ] ⋅ x
end

dim = 2
sampler = ZigZag(dim, ∇U; refresh_rate=0.0)

N_sk, xinit, vinit = 1_000, zeros(dim), ones(dim)

output_ZZ = sample_skeleton(sampler, N_sk, xinit, vinit, seed=20250428)
x_ZZ = output_ZZ.X
v_ZZ = output_ZZ.V
t_ZZ = output_ZZ.t

anim_traj(output_ZZ, 100; plot_start=50, filename="ZZS.gif", color="#E95420", scatter_color="#78C2AD", background="#F0F1EB", title="Zig-Zag on an Anisotropic Target", xlims=(-5, 5), ylims=(-5, 5))

In [4]:
#| eval: false
#| code-fold: true

using PDMPFlux, Plots, ProgressBars, LinearAlgebra, StatsPlots

const dim = 100

@inline function ∇U(x::AbstractVector)
  return x
end

@inline function ∇U_Boomerang(x::AbstractVector)
  return ones(dim)
end

function h_estimate_online(
  sampler::PDMPFlux.AbstractPDMP,
  T_end::Float64,
  xinit::Vector{Float64},
  vinit::Vector{Float64};
  seed::Union{Int, Nothing}=nothing,
)::Float64
  if !(isfinite(T_end)) || T_end < 0
    throw(ArgumentError("T_end must be finite and non-negative. Current value: $T_end"))
  end
  d = length(xinit)
  if d == 0
    throw(ArgumentError("xinit must be non-empty"))
  end
  if T_end == 0.0
    return 0.0
  end

  state = PDMPFlux.init_state(sampler, xinit, vinit, seed)

  estimate = 0.0

  # 現在のイベント点（post-jump）を保持し、次イベントまでの区間寄与を足す
  t_prev = state.t
  x_prev = copy(state.x)
  v_prev = copy(state.v)

  while t_prev < T_end
    PDMPFlux.get_event_state!(state, sampler)  # mutates state in-place; advances state.t to next accepted event
    t_next = state.t

    if t_next <= T_end
      Δt = t_next - t_prev
      # 区間 [t_prev, t_next] は (x_prev, v_prev) からの flow
      nx = dot(x_prev, x_prev)
      xv = dot(x_prev, v_prev)
      estimate += nx * Δt + (Δt^3) / 3 + (Δt^2) * xv

      t_prev = t_next
      copyto!(x_prev, state.x)
      copyto!(v_prev, state.v)
    else
      # overshoot: 最後は flow で t=T_end の点を作ってそこで打ち切る
      Δt = T_end - t_prev
      nx = dot(x_prev, x_prev)
      xv = dot(x_prev, v_prev)
      estimate += nx * Δt + (Δt^3) / 3 + (Δt^2) * xv
      break
    end
  end

  return (estimate - d * T_end) / sqrt(d) / T_end  # CAUTION: inprecise denominator (intentionally)
end

function Experiment_once()
  ZZS = ZigZag(dim, ∇U; refresh_rate=0.0)
  T, xinit, vinit = Float64(1000), randn(dim), ones(dim)
  t_ZZ = @elapsed output_ZZ = h_estimate_online(ZZS, T, xinit, vinit)

  BPS_sampler = BPS(dim, ∇U; refresh_rate=1.4)
  vinit = randn(dim)
  vinit = vinit ./ norm(vinit)
  t_BPS = @elapsed output_BPS = h_estimate_online(BPS_sampler, T, xinit, vinit)

  vinit = randn(dim)
  vinit = vinit ./ norm(vinit)
  FECMC = ForwardECMC(dim, ∇U)
  t_ForwardECMC = @elapsed output_ForwardECMC = h_estimate_online(FECMC, T, xinit, vinit)
  return [abs2(output_ZZ) * t_ZZ, abs2(output_BPS) * t_BPS, abs2(output_ForwardECMC) * t_ForwardECMC]
end

function Experiment(iter::Int=100)
  results = Matrix{Float64}(undef, iter, 3)
  for i in ProgressBar(1:iter)
    results[i,:] = Experiment_once()
  end
  return results
end

results = Experiment(100)

iter  = size(results, 1)
labels = ["ZZS", "BPS", "FECMC"]
groups = repeat(labels, inner=iter)
groups = categorical(groups; ordered=true, levels=labels)
boxplot(groups, vec(results);
    group = groups,
    bar_width = 0.7,
    legend = :bottomleft,
    xlabel = "Sampler",
    ylabel = "Squared Error × Computational Time",
    yscale = :log10)